<img src="./res/webrtc.png">

# Prototyping with WebRTC

 - **[WebRTC](https://developer.mozilla.org/en-US/docs/Web/API/WebRTC_API) - Open Standard - Realtime Communication Voice & Video**
 
 ## Key Interfaces
 - [RTCPeerConnection](https://developer.mozilla.org/en-US/docs/Web/API/RTCPeerConnection)
 - [RTCSessionDescription](https://developer.mozilla.org/en-US/docs/Web/API/RTCSessionDescription)
 - [RTCIceCandidate - Interactive Connectivity Establishment](https://developer.mozilla.org/en-US/docs/Web/API/RTCIceCandidate)

<div>
    <img src="./res/ths_setup1.png" height=400 width=550>
    <img src="./res/ths_setup2.png" height=400 width=900>
</div>

## Emphasis

✔ **A distributed architectural prototype in context of IoT application scenarios**

✔ **Pub/Sub oriented loosely coupled system**

✔ **Container-oriented fast prototyping**

✔ **Aspect of Security**

🟠 **Efficiency in processing and transmission of video frames**

## Constraints

- **Initiation of the P2P connection request by the subscriber**

- **One way media streams**

# Basic Architecture

<img src="./res/ths_base_arch.png">

### Limitations

- **Strong Coupling - Change on one end leads to a change on other**

- **Security Concerns**

# Revised Architecture - Signalling Server as Mediator

<img src="./res/ths_revised_arch.png">

<img src="./res/ths_seq_diag.png">

#### Subsequent Improvement to the architecture

- **Adoption event driven approach for the signalling server**

# Orchestration of Containerized Services

### Subscriber Side (Browser-based Web Application)
```yaml
version : ’3’ 
services :
    traefik : 
        privileged : true
        command : 
            −−api −−docker −−docker.domain="docker.localhost" 
            −−docker.endpoint=unix:///var/run/docker.sock \ 
            −−docker.watch=true −−docker.exposedbydefault="false"
        container_name: traefik 
        image: traefik
        volumes :
            − /var/run/docker.sock:/var/run/docker.sock 
        ports :
            − 9999:80
            − 8080:8080 
    signaling :
        container_name : signaling 
        build : ./ signaling
        expose :
            − 9999 
        labels :
            − "traefik.frontend.rule=Host:docker.localhost;PathPrefix:/signaling/ subscribe;"
            − "traefik.frontend.entryPoints=http" − "traefik.port=9999"
            − "traefik.enable=true"
    subscriber : 
        container_name : subscriber
        build : ./ client 
        depends_on :
            − signaling 
        expose :
            − 9999 
        labels :
            −"traefik.frontend.rule=Host:docker.localhost;PathPrefix:/client,/socket.io ,/receive_publisher_answer;"
            − "traefik.frontend.entryPoints=http" − "traefik.port=9999"
            − "traefik.enable=true"
        command : [ "./wait-for-it.sh" , "signaling:9999" , "--" , "npm" , "start" ]
```

### Publisher Side (Raspberry Py 3B with Cam)

```yaml
version : ’3’ 
services :
    recorder :
        container_name : recorder 
        build : . / recorder 
        devices :
            − /dev/video0:/dev/video0 
        volumes :
            − record−volume: /record 
    publisher :
        container_name : publisher 
        build : . / publisher 
        depends_on:
            - recorder
        volumes :
            − record−volume:/record 
        ports :
            − 8888:8888
        command : [ "python" , "/usr/src/publisher.py" , "--port" , "8888" , "-v" ]
volumes : 
    record−volume :
```

### Consideration

- Is event streaming a viable option for high performance data processing pipeline ?
- Is p2p communication is a viable approach for high performance data processing pipeline ?
